In [114]:
from src.utils import *
from src.graph_model import *
from src.owl2vec import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

In [116]:
#dataset_name = 'family'
#dataset_name = 'pizza'
dataset_name = 'OWL2DL-1'

In [117]:
experiments = get_experimets(dataset_name)

In [ ]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']
          
    model = OWL2Vec(file_name         = file_name,
                    dataset_name      = dataset_name,
                    kge_model         = 'transe',
                    emb_dim           = 256,
                    margin            = 0.1,
                    weight_decay      = 0.0,
                    batch_size        = 4096*8,
                    lr                = 0.0001,
                    num_negs          = 4,
                    test_batch_size   = 32,
                    epochs            = 500,
                    device            = device,
                    seed              = 42,
                    initial_tolerance = 5)

    model.train()
    
    print(f'{file_name}:')
    metrics_subsumption, metrics_membership = model.test()
    print()
    save_results(metrics_subsumption, metrics_membership, f'models/results/owl2vec/{file_name}.txt')